In [ ]:
import os
import pickle
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LogNorm
from copy import copy

In [ ]:
filepath = "/DATA_MASTER_PATH/23_hike_pinunu-background/2308_zoptical-zanalyze_final_vars/hist_norm_conv1plus_withgencls_cut0p5032/"

def filesel(s):
    #sel = ("_0." in s)
    sel = ("sig_mal" in s) | ("bkg_2p0_mal" in s)
    #sel = True
    return sel

files_2pi = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("2p0_mal" in s))]
files_sig = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("sig_mal" in s))]
files_lam = [s for s in os.listdir(filepath) if ((os.path.isfile(os.path.join(filepath, s))) & filesel(s) & ("lambda" in s))]

files_bkg = files_2pi

In [ ]:
def histsum(files):
# 1st index ("hist_0/1") identifies the BDT, 2nd index (0/1) the predicted class

    n_decays = 0

    for ifile, file, in enumerate(files):
        with open(filepath+file, "rb") as f:
            hists = pickle.load(f)
            
        n_decays_0 = n_decays
        for i in range(1, 10):
            if "_%dM_"%(i) in file:
                n_decays += i*1e6
            if "_%dM_"%(10*i) in file:
                n_decays += i*10e6
            if "_%dM_"%(100*i) in file:
                n_decays += i*100e6
            if "_%dG_"%(i) in file:
                n_decays += i*1e9
            if "_%dG_"%(10*i) in file:
                n_decays += i*10e9
        print("opened %d/%d, with %d new decays" % (ifile+1, len(files), n_decays-n_decays_0))

        if ifile==0:
            hists_tot = {
                "hist_0" : {
                    0 : list(hists["hist_0"][0]),
                    1 : list(hists["hist_0"][1]),
                },
                "hist_1" : {
                    0 : list(hists["hist_1"][0]),
                    1 : list(hists["hist_1"][1]),
                },
            }

        else:
            for i0 in ("hist_0", "hist_1"):
                for i1 in (0, 1):
                    hists_tot[i0][i1][0] += hists[i0][i1][0]

    return hists_tot, n_decays
    
hists_bkg, n_decays_bkg = histsum(files_bkg)
hists_sig, n_decays_sig = histsum(files_sig)

In [ ]:
print("total signal (background) decays are %E (%E)" % (n_decays_sig, n_decays_bkg))

n_decays_sig_loi, n_decays_bkg_loi = 15483, 446e9
print("in the LoI, %E (%E) signal (background events)" % (n_decays_sig_loi, n_decays_bkg_loi))

In [ ]:
hist_id = "hist_0"

extent = [
    min(hists_sig[hist_id][0][1]), max(hists_sig[hist_id][0][1]),
    min(hists_sig[hist_id][0][2]), max(hists_sig[hist_id][0][2])
]

blog = True

bnorm = False

box_edges = (280-150, 350-150, 0.14, 10)

box = patches.Rectangle(
    (box_edges[0], box_edges[2]), box_edges[1]-box_edges[0], box_edges[3]-box_edges[2],
    linewidth=1, edgecolor='0.5', facecolor='none', label="LoI fiducial box"
)

In [ ]:
print("in total:")

print("--> %E (%E) true signal (background) events" % (
    np.sum(hists_sig[hist_id][0][0]+hists_sig[hist_id][1][0]),
    np.sum(hists_bkg[hist_id][0][0]+hists_bkg[hist_id][1][0])
))

print("--> %E (%E) predicted signal (background) events" % (
    np.sum(hists_sig[hist_id][1][0]+hists_bkg[hist_id][1][0]),
    np.sum(hists_sig[hist_id][0][0]+hists_bkg[hist_id][0][0])
))

print("--> %E truly predicted positive events (%.4f%% of true positive, %.4f%% of pred. positive)" % (
    np.sum(hists_sig[hist_id][1][0]),
    np.sum(hists_sig[hist_id][1][0]) / np.sum(hists_sig[hist_id][0][0]+hists_sig[hist_id][1][0]) *100,
    np.sum(hists_sig[hist_id][1][0]) / np.sum(hists_sig[hist_id][1][0]+hists_bkg[hist_id][1][0]) *100
))

print("--> %E truly predicted negative events (%.4f%% of true negative, %.4f%% of pred. negative)" % (
    np.sum(hists_bkg[hist_id][0][0]),
    np.sum(hists_bkg[hist_id][0][0]) / np.sum(hists_bkg[hist_id][0][0]+hists_bkg[hist_id][1][0]) *100,
    np.sum(hists_bkg[hist_id][0][0]) / np.sum(hists_sig[hist_id][0][0]+hists_bkg[hist_id][0][0]) *100
))

print("--> %E falsely predicted positive events (%.4f%% of true negative, %.4f%% of pred. positive)" % (
    np.sum(hists_bkg[hist_id][1][0]),
    np.sum(hists_bkg[hist_id][1][0]) / np.sum(hists_bkg[hist_id][0][0]+hists_bkg[hist_id][1][0]) *100,
    np.sum(hists_bkg[hist_id][1][0]) / np.sum(hists_sig[hist_id][1][0]+hists_bkg[hist_id][1][0]) *100
))

print("--> %E falsely predicted negative events (%.4f%% of true positive, %.4f%% of pred. negative)" % (
    np.sum(hists_sig[hist_id][0][0]),
    np.sum(hists_sig[hist_id][0][0]) / np.sum(hists_sig[hist_id][0][0]+hists_sig[hist_id][1][0]) *100,
    np.sum(hists_sig[hist_id][0][0]) / np.sum(hists_sig[hist_id][0][0]+hists_bkg[hist_id][0][0]) *100
))

print("--> truly predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_sig[hist_id][1][0]) * (n_decays_sig_loi/n_decays_sig)
))
print("--> falsely predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_bkg[hist_id][1][0]) * (n_decays_bkg_loi/n_decays_bkg)
))

In [ ]:
print("in final box (cut on p_T only):")

bins_x = hists_sig[hist_id][0][1][:-1]+0.5*(hists_sig[hist_id][0][1][1]-hists_sig[hist_id][0][1][0])
bins_y = hists_sig[hist_id][0][2][:-1]+0.5*(hists_sig[hist_id][0][2][1]-hists_sig[hist_id][0][2][0])
bool_box_x = (bins_x>-9999) & (bins_x<9999)
bool_box_y = (bins_y>box_edges[2]) & (bins_y<box_edges[3])

print("--> %E (%E) true signal (background) events" % (
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E (%E) predicted signal (background) events" % (
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E truly predicted positive events (%.4f%% of true positive, %.4f%% of pred. positive)" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E truly predicted negative events (%.4f%% of true negative, %.4f%% of pred. negative)" % (
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted positive events (%.4f%% of true negative, %.4f%% of pred. positive)" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted negative events (%.4f%% of true positive, %.4f%% of pred. negative)" % (
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> truly predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_sig_loi/n_decays_sig)
))
print("--> falsely predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_bkg_loi/n_decays_bkg)
))

In [ ]:
print("in final box (as in LoI):")

bins_x = hists_sig[hist_id][0][1][:-1]+0.5*(hists_sig[hist_id][0][1][1]-hists_sig[hist_id][0][1][0])
bins_y = hists_sig[hist_id][0][2][:-1]+0.5*(hists_sig[hist_id][0][2][1]-hists_sig[hist_id][0][2][0])
bool_box_x = (bins_x>box_edges[0]) & (bins_x<box_edges[1])
bool_box_y = (bins_y>box_edges[2]) & (bins_y<box_edges[3])

print("--> %E (%E) true signal (background) events" % (
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E (%E) predicted signal (background) events" % (
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E truly predicted positive events (%.4f%% of true positive, %.4f%% of pred. positive)" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E truly predicted negative events (%.4f%% of true negative, %.4f%% of pred. negative)" % (
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted positive events (%.4f%% of true negative, %.4f%% of pred. positive)" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted negative events (%.4f%% of true positive, %.4f%% of pred. negative)" % (
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> truly predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_sig_loi/n_decays_sig)
))
print("--> falsely predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_bkg_loi/n_decays_bkg)
))

In [ ]:
shift_fv = -30
shift_pt = 0.00

print("in final box (FV shifted by %d, cut on p_t shifted by %.2f):" % (shift_fv, shift_pt))

bins_x = hists_sig[hist_id][0][1][:-1]+0.5*(hists_sig[hist_id][0][1][1]-hists_sig[hist_id][0][1][0])
bins_y = hists_sig[hist_id][0][2][:-1]+0.5*(hists_sig[hist_id][0][2][1]-hists_sig[hist_id][0][2][0])
bool_box_x = (bins_x>(box_edges[0]+shift_fv)) & (bins_x<(box_edges[1]+shift_fv))
bool_box_y = (bins_y>(box_edges[2]+shift_pt)) & (bins_y<(box_edges[3]+shift_pt))

print("--> %E (%E) true signal (background) events" % (
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E (%E) predicted signal (background) events" % (
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ),
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    )
))

print("--> %E truly predicted positive events (%.4f%% of true positive, %.4f%% of pred. positive)" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E truly predicted negative events (%.4f%% of true negative, %.4f%% of pred. negative)" % (
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted positive events (%.4f%% of true negative, %.4f%% of pred. positive)" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> %E falsely predicted negative events (%.4f%% of true positive, %.4f%% of pred. negative)" % (
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]),
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100,
    np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]) /\
    np.sum(
        hists_sig[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]+\
        hists_bkg[hist_id][0][0][np.ix_(bool_box_x, bool_box_y)]
    ) *100
))

print("--> truly predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_sig_loi/n_decays_sig)
))
print("--> falsely predicted positive events rescaled to LoI statistics: %f" % (
    np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x, bool_box_y)]) * (n_decays_bkg_loi/n_decays_bkg)
))

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5), num=hist_id)

ax = axs[0]
ax.set_title("true background")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_bkg[hist_id][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="pred. as background")

data = hists_bkg[hist_id][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="pred. as signal")

ax = axs[1]
ax.set_title("true signal")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_sig[hist_id][0][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_sig[hist_id][1][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5), num=hist_id)

pred_class = 0
ax = axs[pred_class]
ax.set_title("predicted as background")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_bkg[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="true background")

data = hists_sig[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="true signal")

pred_class = 1
ax = axs[pred_class]
ax.set_title("predicted as signal")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_bkg[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_sig[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

axs[0].legend(loc="upper left")
fig.tight_layout()

In [ ]:
print("in final box (in custom-shaped box):")

bins_x = hists_sig[hist_id][0][1][:-1]+0.5*(hists_sig[hist_id][0][1][1]-hists_sig[hist_id][0][1][0])
bins_y = hists_sig[hist_id][0][2][:-1]+0.5*(hists_sig[hist_id][0][2][1]-hists_sig[hist_id][0][2][0])
bool_box_x = []
bool_box_y = []

def line0(x):
    x0, x1, y0, y1 = 100, 125, 0.25, 0.130
    m = (y1-y0)/(x1-x0)
    q = y1-m*x1
    return m*x+q
def line1(x):
    x0, x1, y0, y1 = 125, 150, 0.130, 0.140
    m = (y1-y0)/(x1-x0)
    q = y1-m*x1
    return m*x+q
def line2(x):
    x0, x1, y0, y1 = 150, 200, 0.140, 0.25
    m = (y1-y0)/(x1-x0)
    q = y1-m*x1
    return m*x+q
for dx in np.arange(115, 190, 0.01):
    line = line0 if dx<125 else (line1 if dx<150 else line2)
    bool_box_x.append((bins_x>dx) & (bins_x<(dx+0.01)))
    bool_box_y.append((bins_y>line(dx)) & (bins_y<box_edges[3]))

print("--> %E (%E) true signal (background) events" % (
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ),
    np.sum(
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    )
))

print("--> %E (%E) predicted signal (background) events" % (
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ),
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    )
))

print("--> %E truly predicted positive events (%.4f%% of true positive, %.4f%% of pred. positive)" % (
    np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]),
    np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100,
    np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100
))

print("--> %E truly predicted negative events (%.4f%% of true negative, %.4f%% of pred. negative)" % (
    np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]),
    np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100,
    np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100
))

print("--> %E falsely predicted positive events (%.4f%% of true negative, %.4f%% of pred. positive)" % (
    np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]),
    np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100,
    np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100
))

print("--> %E falsely predicted negative events (%.4f%% of true positive, %.4f%% of pred. negative)" % (
    np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]),
    np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100,
    np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) /\
    np.sum(
        np.sum([np.sum(hists_sig[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])+\
        np.sum([np.sum(hists_bkg[hist_id][0][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))])
    ) *100
))

print("--> truly predicted positive events rescaled to LoI statistics: %f" % (
    np.sum([np.sum(hists_sig[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) * (n_decays_sig_loi/n_decays_sig)
))
print("--> falsely predicted positive events rescaled to LoI statistics: %f" % (
    np.sum([np.sum(hists_bkg[hist_id][1][0][np.ix_(bool_box_x[i], bool_box_y[i])]) for i in range(len(bool_box_x))]) * (n_decays_bkg_loi/n_decays_bkg)
))

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5), num=hist_id)

pred_class = 0
ax = axs[pred_class]
ax.set_title("predicted as background")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_bkg[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")
ax.plot((-1, -1), color="r", lw=0, marker="o", label="true background")

data = hists_sig[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)
ax.plot((-1, -1), color="b", lw=0, marker="o", label="true signal")

pred_class = 1
ax = axs[pred_class]
ax.set_title("predicted as signal")
ax.set_xlim((extent[0], extent[1]))
ax.set_ylim((extent[2], extent[3]))
ax.add_patch(copy(box))

data = hists_bkg[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Reds")

data = hists_sig[hist_id][pred_class][0].T
hist = np.ma.masked_where(data == 0, data) / (np.sum(data) if bnorm else 1)
ax.matshow(hist, origin="lower", extent=extent, aspect="auto", norm=LogNorm() if blog else None, cmap="Blues", alpha=0.6 if blog else 0.9)

for ax in axs:
    #ax.axvline(100, color="k", ls=":")
    #ax.axvline(150, color="k", ls=":")
    #ax.axvline(200, color="k", ls=":")
    ax.plot((115, 115), (line0(115), 0.4), color="green", label="custom box")
    ax.plot((115, 125), (line0(115), line0(125)), color="green")
    ax.plot((125, 150), (line1(125), line1(150)), color="green")
    ax.plot((150, 190), (line2(150), line2(190)), color="green")
    ax.plot((190, 190), (line2(190), 0.4), color="green")

axs[0].legend(loc="upper left")
fig.tight_layout()